In [1]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')
from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.neural import GeneClassifier
from models.lib.data import *
from models.lib.neural import *
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *

Let's do some simple preprocessing for training, and then test our library

In [2]:
# le = LabelEncoder()

# labels = pd.read_csv('../data/retina/retina_labels.tsv', sep='\t')
# labels = labels[labels['CellType'].isin(['retinal rod cell type B', 'retinal rod cell type A'])]
# labels = labels[labels['CellType'] != 'unannotated']
# labels['class_label'] = le.fit_transform(labels['CellType'])

# labels.index.name = 'cell'
# # labels = labels.iloc[0:5000, :]
# labels.to_csv('../data/retina/retina_labels_numeric.csv')

# # label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
# # # label_df

# # labels.to_csv('../data/retina/retina_labels_numeric.csv')

# labels = pd.read_csv('../data/retina/retina_labels_numeric.csv')
# labels.loc[:, 'class_label']

In [3]:
labels = pd.read_csv('../data/retina/retina_labels_numeric.csv')
labels.loc[:, 'class_label'].value_counts()

10    6142
9     5924
3      714
0      612
7      583
4      566
12     456
5      421
6      369
1      281
11     173
2      142
8       63
Name: class_label, dtype: int64

In [4]:
module = DataModule(
    datafiles=['../data/retina/retina_T.csv'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
#     sep=',',
    batch_size=16,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    normalize=True,
)

In [6]:
module.setup()
next(iter(module.trainloader))
compute_class_weights(['../data/retina/retina_labels_numeric.csv'], 'class_label')

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(


IN TOTALCLSASWIRIGHTS sep=',' GOD FUCKIGN DAMMIOT


tensor([ 2.0671,  4.5021,  8.9090,  1.7718,  2.2351,  3.0049,  3.4284,  2.1699,
        20.0806,  0.2136,  0.2060,  7.3126,  2.7743])

In [ ]:
tabnetmodel = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.2,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.StepLR,
        'step_size': 1e-5,
    },
    metrics={
        'confusionmatrix': confusion_matrix,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
)

wandb_logger = WandbLogger(
    project=f"tabnet-classifer-sweep",
    name='local-retina-model'
)

trainer = pl.Trainer(
    gpus=(1 if torch.cuda.is_available() else 0),
    auto_lr_find=False,
    logger=wandb_logger,
    max_epochs=100,
)

In [ ]:
trainer.fit(tabnetmodel, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 2.5 M 
-----------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.916     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:453: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:219: UserWarning: You called `self.log('val_confusionmatrix', ...)` in your `validation_step` but the value needs to be floating point. Converting it to torch.float32.
  warnin

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
orig = an.read_h5ad('../data/retina/retina.h5ad')

In [ ]:
t = GeneExpressionData(
    filename='../data/retina/retina_T.csv',
    labelname='../data/retina/retina_labels_numeric.csv',
    index_col='cell',
    class_label='numeric_class',
)

In [ ]:
t.columns.__len__()

In [ ]:
from bigcsv import BigCSV

converter = BigCSV(
    file='../data/retina/retina_T.csv',
    outfile='../data/retina/retina_test.h5ad',
    save_chunks=True,
    chunkfolder='./chunks'
)

converter.to_h5ad()